In [1]:
import torch
import os.path as osp
from math import sqrt

import torch
import torch.nn.functional as F
from rdkit import Chem

from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn.models import AttentiveFP

import os
import os.path as osp
import re

import torch
import torch.nn.functional as F
from rdkit import Chem
from torch_geometric.data import (InMemoryDataset, Data, download_url,
                                  extract_gz)
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn.models import AttentiveFP

from model import GNN,GNN_graphpred
from dataset import LSSInhibitor,GenAtomFeatures,GenAttentiveFeatures
from loss import ada_batch_all_triplet_loss

device = 'cuda:6'
batch_size = 16
epochs = 200
lr = 0.0001
decay = 1e-3
num_layer = 5
emb_dim = 300
dropout_ratio = 0
JK = 'last'
dataset = 'EAAT3'
output_model_file = ''
gnn_type = 'gin'
seed = 0
num_workers = 8
mode = 'ada_batch_all_triplet_loss'   #ada_batch_all_triplet_loss,ada_batch_hard_triplet_loss,triplet_loss
feature_type = 'custom'  #random,onehot,custom,pseudo
graph_pooling = 'set2set2' #mean,last,sum,set2set,atention


def train():
    total_loss = total_examples = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        '''
        data.x = data.x.to(torch.float32)
        data.edge_index=data.edge_index.to(torch.long)
        data.edge_attr=data.edge_attr.to(torch.float32)
        data.batch=data.batch.to(torch.long)
        '''
        #out = model(float(data.x), data.edge_index.to(torch.long), data.edge_attr.to(torch.float32), data.batch.to(torch.long))
        emb,pre = model(data.x, data.edge_index, data.edge_attr, data.batch)

        #loss = F.mse_loss(out, data.y)
        loss = ada_batch_all_triplet_loss(embeddings=emb, labels=data.y, prediction=pre, device=device, minv=minv, maxv=maxv, weight=0.5, cliff=2,squared=False)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
        total_examples += data.num_graphs
    return sqrt(total_loss / total_examples)


@torch.no_grad()
def test(loader):
    mse = []
    for data in loader:
        data = data.to(device)
        emb,pre = model(data.x.to(torch.float32), data.edge_index, data.edge_attr, data.batch)
        pre = pre*(maxv-minv)+minv
        mse.append(F.mse_loss(pre, data.y, reduction='none').cpu())
        #print('mse:',mse)
    return float(torch.cat(mse, dim=0).mean().sqrt())


for dataset_name in LSSInhibitor.names.keys():
    
    print(dataset_name + "\n")
    
    path = './data'
    
    # use the attentiveFP node and edge features during the mol-2-graph transoformation
    dataset = LSSInhibitor(path, name=dataset_name, pre_transform=GenAtomFeatures('custom')).shuffle()
    #dataset = LSSInhibitor(path, name=dataset_name, pre_transform=GenAttentiveFeatures()).shuffle()
    #dataset = MoleculeNet(path, name='FreeSolv', pre_transform=GenFeatures()).shuffle()
    print(dataset.data.x.shape)
    print(dataset.__dict__)
    
    minv = 1e12
    maxv = -1e12
    for i in dataset:
        if i.y<minv:
            minv = i.y.item()
        if i.y>maxv:
            maxv = i.y.item()
    print('minv:',minv) 
    print('maxv:',maxv)
    
    #batch_size = 8
    
    # train, valid, test splitting
    N = len(dataset) // 5
    val_dataset = dataset[:N]
    test_dataset = dataset[N:2 * N]
    train_dataset = dataset[2 * N:]
    
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    device = torch.device(device if torch.cuda.is_available() else 'cpu')

    model = GNN_graphpred(num_layer, emb_dim, num_tasks = 1, JK = JK, drop_ratio =dropout_ratio,feature_type = feature_type,graph_pooling = graph_pooling, gnn_type = gnn_type).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                 weight_decay=decay)

    for epoch in range(epochs):
        train_rmse = train()
        val_rmse = test(val_loader)
        test_rmse = test(test_loader)
        print(f'Epoch: {epoch:03d}, Loss: {train_rmse:.4f} Val: {val_rmse:.4f} '
              f'Test: {test_rmse:.4f}')

mglur2

torch.Size([791, 115])
{'name': 'mglur2', 'root': 'data', 'transform': None, 'pre_transform': <dataset.GenAtomFeatures object at 0x7fa610554430>, 'pre_filter': None, '_indices': [7, 23, 3, 25, 4, 19, 17, 22, 16, 5, 2, 11, 8, 18, 12, 13, 9, 6, 24, 20, 10, 0, 21, 14, 1, 15], 'data': Data(x=[791, 115], edge_index=[2, 1734], edge_attr=[1734, 7], y=[26, 1], smiles=[26]), 'slices': defaultdict(<class 'dict'>, {'x': tensor([  0,  33,  54,  74, 102, 135, 169, 203, 230, 251, 285, 320, 348, 376,
        410, 444, 472, 506, 540, 576, 611, 632, 661, 695, 730, 764, 791]), 'edge_index': tensor([   0,   72,  118,  162,  224,  296,  370,  444,  504,  552,  626,  702,
         764,  826,  900,  974, 1036, 1110, 1184, 1262, 1338, 1386, 1450, 1524,
        1600, 1674, 1734]), 'edge_attr': tensor([   0,   72,  118,  162,  224,  296,  370,  444,  504,  552,  626,  702,
         764,  826,  900,  974, 1036, 1110, 1184, 1262, 1338, 1386, 1450, 1524,
        1600, 1674, 1734]), 'y': tensor([ 0,  1,  2

In [1]:
import torch
import os.path as osp
from math import sqrt

import torch
import torch.nn.functional as F
from rdkit import Chem

from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn.models import AttentiveFP

import os
import os.path as osp
import re

import torch
import torch.nn.functional as F
from rdkit import Chem
from torch_geometric.data import (InMemoryDataset, Data, download_url,
                                  extract_gz)
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn.models import AttentiveFP

from model import GNN,GNN_graphpred
from dataset import LSSInhibitor,GenAtomFeatures,GenAttentiveFeatures
from loss import ada_batch_all_triplet_loss

device = 'cuda:7'
batch_size = 16
epochs = 200
lr = 0.0001
decay = 0
num_layer = 5
emb_dim = 300
dropout_ratio = 0
JK = 'last'
dataset = 'EAAT3'
output_model_file = ''
gnn_type = 'gin'
seed = 0
num_workers = 8
mode = 'ada_batch_all_triplet_loss'   #ada_batch_all_triplet_loss,ada_batch_hard_triplet_loss,triplet_loss
feature_type = 'custom'  #random,onehot,custom,pseudo
graph_pooling = 'set2set2' #mean,last,sum,set2set,atention


def train():
    total_loss = total_examples = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        '''
        data.x = data.x.to(torch.float32)
        data.edge_index=data.edge_index.to(torch.long)
        data.edge_attr=data.edge_attr.to(torch.float32)
        data.batch=data.batch.to(torch.long)
        '''
        #out = model(float(data.x), data.edge_index.to(torch.long), data.edge_attr.to(torch.float32), data.batch.to(torch.long))
        emb,pre = model(data.x, data.edge_index, data.edge_attr, data.batch)

        #loss = F.mse_loss(out, data.y)
        loss = ada_batch_all_triplet_loss(embeddings=emb, labels=data.y, prediction=pre, device=device, minv=minv, maxv=maxv, weight=0.5, cliff=2,squared=False)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
        total_examples += data.num_graphs
    return sqrt(total_loss / total_examples)


@torch.no_grad()
def test(loader):
    mse = []
    for data in loader:
        data = data.to(device)
        emb,pre = model(data.x.to(torch.float32), data.edge_index, data.edge_attr, data.batch)
        pre = pre*(maxv-minv)+minv
        mse.append(F.mse_loss(pre, data.y, reduction='none').cpu())
        #print('mse:',mse)
    return float(torch.cat(mse, dim=0).mean().sqrt())


for dataset_name in LSSInhibitor.names.keys():
    
    print(dataset_name + "\n")
    
    path = './data'
    
    # use the attentiveFP node and edge features during the mol-2-graph transoformation
    dataset = LSSInhibitor(path, name=dataset_name, pre_transform=GenAtomFeatures('custom')).shuffle()
    print(dataset[0])
    print(dataset[1:5])
    
    dt = sorted(dataset,key = lambda data:data.y )
    dt_dict={}
    for i in range(5):
        dt_dict[i] = {'train':[],'test':[]}
    
    for i in range(len(dt)):
        tmp = i % 5
        for j in range(5):
            if tmp == j:
                dt_dict[j]['test'].append(dt[i])
            else:
                dt_dict[j]['train'].append(dt[i])
  
    print('dict',dt_dict)
    print(len(dt_dict[0]['train']))
    print(len(dt_dict[0]['test']))
    train_data = DataLoader(dt_dict[0]['train'], batch_size=batch_size, shuffle=True)
    test_data = DataLoader(dt_dict[0]['test'], batch_size=batch_size, shuffle=False)
    print(len(train_data))
    print(len(test_data))
    exit(0)
    #dataset = LSSInhibitor(path, name=dataset_name, pre_transform=GenAttentiveFeatures()).shuffle()
    #dataset = MoleculeNet(path, name='FreeSolv', pre_transform=GenFeatures()).shuffle()
    print(dataset.data.x.shape)
    print(dataset.__dict__)
    
    
    minv = 1e12
    maxv = -1e12
    for i in dataset:
        if i.y<minv:
            minv = i.y.item()
        if i.y>maxv:
            maxv = i.y.item()
    print('minv:',minv) 
    print('maxv:',maxv)
    
    #batch_size = 8
    
    # train, valid, test splitting
    N = len(dataset) // 5
    val_dataset = dataset[:N]
    test_dataset = dataset[N:2 * N]
    train_dataset = dataset[2 * N:]
    
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    device = torch.device(device if torch.cuda.is_available() else 'cpu')

    model = GNN_graphpred(num_layer, emb_dim, num_tasks = 1, JK = JK, drop_ratio =dropout_ratio,feature_type = feature_type,graph_pooling = graph_pooling, gnn_type = gnn_type).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                 weight_decay=decay)

    for epoch in range(epochs):
        train_rmse = train()
        val_rmse = test(val_loader)
        test_rmse = test(test_loader)
        print(f'Epoch: {epoch:03d}, Loss: {train_rmse:.4f} Val: {val_rmse:.4f} '
              f'Test: {test_rmse:.4f}')

mglur2

Data(x=[34, 115], edge_index=[2, 74], edge_attr=[74, 7], y=[1, 1], smiles='CC[C@@](O)(c1cn(Cc2ccc3c(-c4ccc(F)cc4)cc(C(N)=O)nc3c2)nn1)C(F)(F)F')
mGluR2(4)
dict {0: {'train': [Data(x=[36, 115], edge_index=[2, 78], edge_attr=[78, 7], y=[1, 1], smiles='CC[C@@](O)(c1cn(Cc2ccc3c(-c4ccc(C(C)C)cc4)cc(C(N)=O)nc3c2)nn1)C(F)(F)F'), Data(x=[34, 115], edge_index=[2, 74], edge_attr=[74, 7], y=[1, 1], smiles='CC[C@@](O)(c1cn(Cc2ccc3c(-c4ccc(F)cc4)cc(C(C)=O)nc3c2)nn1)C(F)(F)F'), Data(x=[27, 115], edge_index=[2, 60], edge_attr=[60, 7], y=[1, 1], smiles='NC(=O)c1cc(-c2ccc(F)cc2)c2ccc(CN3CCOCC3)cc2n1'), Data(x=[34, 115], edge_index=[2, 74], edge_attr=[74, 7], y=[1, 1], smiles='CC[C@@](O)(c1cn(Cc2ccc3c(-c4ccc(Cl)cc4)cc(C(N)=O)nc3c2)nn1)C(F)(F)F'), Data(x=[34, 115], edge_index=[2, 74], edge_attr=[74, 7], y=[1, 1], smiles='CC[C@@](O)(c1nnn(Cc2ccc3c(-c4ccc(F)cc4)cc(C(N)=O)nc3c2)n1)C(F)(F)F'), Data(x=[28, 115], edge_index=[2, 62], edge_attr=[62, 7], y=[1, 1], smiles='NC(=O)c1cc(-c2ccc(F)cc2)c2ccc(CN3C

KeyboardInterrupt: 